# Science User Case - Inspecting a Candidate List

Ogle et al. (2016) mined the NASA/IPAC Extragalactic Database (NED) to identify a new type of galaxy: Superluminous Spiral Galaxies. Here's the paper:

Here's the paper: https://ui.adsabs.harvard.edu//#abs/2016ApJ...817..109O/abstract

Table 1 lists the positions of these Super Spirals. Based on those positions, let's create multiwavelength cutouts for each super spiral to see what is unique about this new class of objects.

## 1. Import some Python modules.

In [ ]:
#Suppress unimportant warnings.
import warnings
warnings.filterwarnings("ignore")
#Add the workshop directory to your path.
import sys
sys.path.append('workshop-dev-master')

#Import the workshop utilities.
#import navo_utils.utils
#from navo_utils.image import Image, ImageColumn
#from navo_utils.spectra import Spectra, SpectraColumn
#from navo_utils.cone import Cone
#from navo_utils.tap import Tap
#from navo_utils.utils import astropy_table_from_votable_response

#Import the astropy Table module.
from astropy.table import Table

## 2. Search NED for objects in this paper.

Insert a Code Cell below by clicking on the "Insert" Menu and choosing "Insert Cell Below". Then consult QuickReference.md to figure out how to use astroquery to search NED for all objects in a paper, based on the refcode of the paper. Inspect the resulting astropy table.

In [ ]:
from astroquery.ned import Ned
objects_in_paper = Ned.query_refcode('2016ApJ...817..109O')
objects_in_paper.show_in_notebook()

## 3. Filter the NED results.

The results from NED will include galaxies, but also other kinds of objects. Print the 'Type' column to see the full range of classifications. Next, print the 'Type' of just the first source in the table, in order to determine its data type (since Python 3 distinguishes between strings and byte strings). Finally, use the data type information to filter the results so that we only keep the galaxies in the list.

In [ ]:
objects_in_paper['Type']

In [ ]:
#Let's see whether we are dealing with strings or byte strings
objects_in_paper[0]['Type']

In [ ]:
#Byte strings it is!
galaxies = objects_in_paper[objects_in_paper['Type'] == b'G']

galaxies.show_in_notebook()

## 4. Search the NAVO Registry for image resources.

The paper selected super spirals using WISE, SDSS, and GALEX images. Search the NAVO registry for all image resources, using the 'service_type' search parameter. How many image resources are currently available?

In [ ]:
#from navo_utils.registry import Registry

#image_services = Registry.query(service_type='image')

#print(f'{len(image_services)} result(s) found.')

#image_services

## 5. Search the NAVO Registry for image resources that will allow you to search for AllWISE images.

There are hundreds of image resources...too many to quickly read through. Try adding the 'keyword' search parameter to your registry search, and find the image resource you would need to search the AllWISE images.

In [ ]:
import pyvo as vo

allwise_image_services = vo.regsearch(servicetype='image', keywords=['allwise'])#, 'NASA/IPAC Infrared Science Archive']) #, waveband='uv')


#allwise_image_services = Registry.query(keyword='allwise', service_type='image')

print(f'{len(allwise_image_services)} result(s) found.')

allwise_image_services.to_table()['ivoid', 'short_name', 'res_title']


## 6. Choose the AllWISE image service that you are interested in.

In [ ]:
allwise_image_service = allwise_image_services[0]
allwise_image_service

## 7. Choose one of the galaxies in the NED list.


In [ ]:
from astropy.coordinates import SkyCoord
ra = 3.95892
dec = -10.045109999999999
pos = SkyCoord(ra, dec, unit = 'deg')

## 8. Search for a list of AllWISE images that cover this galaxy.

How many images are returned? Which are you most interested in?

In [ ]:
#from navo_utils.image import Image

#Doing an image query will return a list.
#Each element represents the search results for each position.
#If you enter only one position, there will be only one element in the list.
#Each element in the list can contain many images.

#allwise_image_table=Image.query(service=allwise_image_service,coords=pos,radius=0)

In [ ]:
allwise_image_table = allwise_image_service.search(pos=pos, size=0)
allwise_image_table

## 9. How many images are returned for this one position? Which are you most interested in?

In [ ]:
allwise_images = allwise_image_table.to_table()
allwise_images

## 10. Filter the list of returned AllWISE images so that you are left with only the images taken in band W1 (3.6 micron)

In [ ]:
allwise_w1_images = allwise_images[allwise_images['sia_bp_id'] == b'W1']
allwise_w1_images

## 11. Visualize this AllWISE image.

In [ ]:
#allwise_w1_image = Image.get_fits(allwise_w1_images[0])
from astropy.io import fits
allwise_w1_image = fits.open(allwise_w1_images['sia_url'][0].decode())

In [ ]:
import matplotlib.pyplot as plt
from astropy.wcs import WCS
%matplotlib inline  

fig = plt.figure()

wcs = WCS(allwise_w1_image[0].header)
ax = fig.add_subplot(1, 1, 1, projection=wcs)
ax.imshow(allwise_w1_image[0].data, cmap='gray_r', origin='lower', vmax = 10)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')

## 12. Plot a cutout of the AllWISE image, centered on your position.

Try a 60 arcsecond cutout.

In [ ]:
from astropy.nddata import Cutout2D

size = 60
cutout = Cutout2D(allwise_w1_image[0].data, pos, (size, size), wcs=wcs)
wcs = cutout.wcs

fig = plt.figure()

ax = fig.add_subplot(1, 1, 1, projection=wcs)
ax.imshow(cutout.data, cmap='gray_r', origin='lower', vmax = 10)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')


## 13. Try visualizing a cutout of a GALEX image that covers your position.